In [1]:
import os
import json
import gc
import cv2
import keras
from keras import backend as K
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from keras.layers import concatenate, Input
from keras.layers import *
from keras.optimizers import Adam
from keras.callbacks import Callback, ModelCheckpoint
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

Using TensorFlow backend.


## Load Data

In [2]:
train_df = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')
train_df['ImageId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
train_df['hasMask'] = ~ train_df['EncodedPixels'].isna()
mask_count_df = train_df.groupby('ImageId').agg(np.sum).reset_index()
mask_count_df.sort_values('hasMask', ascending=False, inplace=True)
non_missing_train_idx = mask_count_df[mask_count_df['hasMask'] > 0]

## Utility functions

In [3]:
def mask2rle(img):
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle2mask(rle, input_shape):
    width, height = input_shape[:2]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return mask.reshape(height, width).T

def build_masks(rles, input_shape):
    depth = len(rles)
    masks = np.zeros((*input_shape, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            masks[:, :, i] = rle2mask(rle, input_shape)
    
    return masks

def build_rles(masks):
    width, height, depth = masks.shape
    
    rles = [mask2rle(masks[:, :, i])
            for i in range(depth)]
    
    return rles

## DataGenerator

In [4]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path='../input/severstal-steel-defect-detection/train_images',
                 batch_size=32, dim=(256, 1600), n_channels=1,
                 n_classes=4, random_state=2019, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        self.on_epoch_end()

    def __len__(self):
    #num batches
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
    #generate batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]

        X = self.__generate_X(list_IDs_batch)

        if self.mode == 'fit':
            y = self.__generate_y(list_IDs_batch)
            return X, y
        elif self.mode == 'predict':
            return X
        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')

    def on_epoch_end(self):
    #update batch index
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)

    def __generate_X(self, list_IDs_batch):
    #generate data (input)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))

        # Generate data
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            img_path = f"{self.base_path}/{im_name}"
            img = self.__load_grayscale(img_path)

            # Store samples
            X[i,] = img

        return X

    def __generate_y(self, list_IDs_batch):
    #generate ground truth
        y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)

        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            image_df = self.target_df[self.target_df['ImageId'] == im_name]

            rles = image_df['EncodedPixels'].values
            masks = build_masks(rles, input_shape=self.dim)

            y[i, ] = masks

        return y

    def __load_grayscale(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float32) / 255.
        img = np.expand_dims(img, axis=-1)

        return img

## Create training and validation generators

In [5]:
from sklearn.model_selection import train_test_split
BATCH_SIZE = 16
train_idx, val_idx = train_test_split(
    non_missing_train_idx.index,  # NOTICE DIFFERENCE
    random_state=2019, 
    test_size=0.1
)

train_generator = DataGenerator(
    train_idx, 
    df=mask_count_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4
)

val_generator = DataGenerator(
    val_idx, 
    df=mask_count_df,
    target_df=train_df,
    batch_size=BATCH_SIZE, 
    n_classes=4
)

## Dice Coefficient

In [6]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [7]:
from keras import regularizers
def build_model(width=32, num_classes=4, activation='softmax', img_height=256, img_width=1600):
    def conv_block(x, width, k_reg=False):
        c1 = Conv2D(width, (3, 3), activation='elu', padding='same') (x)
        if(k_reg):
            c1 = Conv2D(width, (3, 3), activation='elu', padding='same', kernel_regularizer=regularizers.l2(0.001)) (c1)
        else:
            c1 = Conv2D(width, (3, 3), activation='elu', padding='same') (c1)
        return c1

    inputs = Input((256, 1600, 1))
    s = Lambda(lambda x: x / 255) (inputs)
    c1 = conv_block(s, width)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = conv_block(p1, width*2, k_reg=True)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = conv_block(p2, width*4, k_reg=True)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = conv_block(p3, width*8, k_reg=True)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = conv_block(p4, width*16)

    u6 = Conv2DTranspose(width*8, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = conv_block(u6, width*8)

    u7 = Conv2DTranspose(width*4, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = conv_block(u7, width*4)

    u8 = Conv2DTranspose(width*2, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = conv_block(u8, width*2)

    u9 = Conv2DTranspose(width, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = conv_block(u9, width)

    outputs = Conv2D(num_classes, (1, 1), activation=activation) (c9)
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [8]:
model = build_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 1600, 1) 0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 256, 1600, 1) 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 1600, 32 320         lambda_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 1600, 32 9248        conv2d_1[0][0]                   
____________________________________________________________________________________________

In [9]:
checkpoint = ModelCheckpoint(
    'unet.h5', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)

history = model.fit_generator(
    train_generator,
    validation_data=val_generator,
    callbacks=[checkpoint],
    use_multiprocessing=False,
    workers=1,
    epochs=3
)

Epoch 1/3
374/374 [==============================] - 459s 1s/step - loss: 1.1665 - dice_coef: 0.0138 - val_loss: 3.9404 - val_dice_coef: 0.0039
Epoch 2/3
 13/374 [>.............................] - ETA: 7:06 - loss: 4.0017 - dice_coef: 0.0031

KeyboardInterrupt: 

In [10]:
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['dice_coef', 'val_dice_coef']].plot()
history_df.to_csv('base_UNET_history.csv')

NameError: name 'history' is not defined

In [11]:
def load_img(code, base, resize=True):
    path = f'{base}/{code}'
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if resize:
        img = cv2.resize(img, (256, 256))
    
    return img

def validate_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [12]:
BATCH_SIZE = 64
sub_df = pd.read_csv('../input/severstal-steel-defect-detection/sample_submission.csv')
sub_df['ImageId'] = sub_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
test_imgs = pd.DataFrame(sub_df['ImageId'].unique(), columns=['ImageId'])
test_imgs.head()

,ImageId
0,004f40c73.jpg
1,006f39c41.jpg
2,00b7fb703.jpg
3,00bbcd9af.jpg
4,0108ce457.jpg


In [ ]:
test_generator = DataGenerator(
        test_imgs.index,
        df=test_imgs,
        shuffle=False,
        mode='predict',
        target_df=test_imgs,
        base_path='../input/severstal-steel-defect-detection/test_images',
        batch_size=BATCH_SIZE,
        n_classes=4
    )

In [16]:
test_df = []

for i in range(0, test_imgs.shape[0], 300):
    batch_idx = list(
        range(i, min(test_imgs.shape[0], i + 300))
    )
    
    test_generator = DataGenerator(
        batch_idx,
        df=test_imgs,
        shuffle=False,
        mode='predict',
        base_path='../input/severstal-steel-defect-detection/test_images',
        target_df=sub_df,
        batch_size=1,
        n_classes=4
    )
    
    batch_pred_masks = model.predict_generator(
        test_generator, 
        workers=1,
        verbose=1,
        use_multiprocessing=False
    )
    
    for j, b in tqdm(enumerate(batch_idx)):
        filename = test_imgs['ImageId'].iloc[b]
        image_df = sub_df[sub_df['ImageId'] == filename].copy()
        
        pred_masks = batch_pred_masks[j, ].round().astype(int)
        pred_rles = build_rles(pred_masks)
        
        image_df['EncodedPixels'] = pred_rles
        test_df.append(image_df)
        
    gc.collect()

300/300 [==============================] - 7s 24ms/step


300it [02:41,  1.85it/s]


300/300 [==============================] - 8s 26ms/step


300it [02:41,  1.86it/s]


300/300 [==============================] - 8s 26ms/step


...

300it [02:41,  1.86it/s]


300/300 [==============================] - 8s 28ms/step


300it [02:42,  1.85it/s]


1/1 [==============================] - 0s 37ms/step


1it [00:00,  1.75it/s]


In [19]:
#test_df = pd.concat(test_df)
print(test_df.shape)
test_df.head()

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [ ]:
test_df[['ImageId_ClassId', 'EncodedPixels']].to_csv('submission.csv', index=False)